# Non-Negative Matrix Factorization 

I have been using matrix factorization and did not see a kernel running it, although it is quite possible I just missed it. This kernel joins the data provided to get the order history of different users, create an order counts matrix from that, and factor it into two lower dimension matrices. The new matrix could be used as additional feature columns and potentially improve LB scores. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from collections import Counter

In [2]:
order_prior = pd.read_csv("../input/order_products__prior.csv")
orders = pd.read_csv("../input/orders.csv")
train = pd.read_csv("../input/order_products__train.csv")
products = pd.read_csv("../input/products.csv")

In [3]:
test = orders[orders['eval_set']=='test']
prior = orders[orders['eval_set']=='prior']
prior.tail()

Match user_id of the prior part of the same orders table. That will give you the order history by user.

In [4]:
test = pd.merge(test, prior[['user_id', 'order_id', 'order_number']], on = 'user_id')
test.head()

Merging with order history from order_prior table, ran on 16gb memory, but timed out here on 8gb so I am running a subset of the data below.

In [5]:
test = pd.merge(test[0:100000], order_prior, left_on = 'order_id_y', right_on = 'order_id')
test['new_order_id'] = test['order_id_x']
test['prior_order_id'] = test['order_id_y']
test = test.drop(['order_id_x', 'order_id_y'], axis = 1)
del [orders, order_prior, train]


Create the product list for each order with filtering in pandas. product_list represents reorders while all represents all items from an order.


In [6]:
product_list = test[test['reordered']==1].groupby(['user_id', 'order_number_x', 'new_order_id'])['product_id'].apply(list)
product_list = pd.DataFrame(product_list.reset_index())
product_list['num_products_reordered'] = product_list.product_id.apply(len)
product_list.head(15)

 The output shows what one user's order history looks like.

In [7]:
test[(test['user_id'] == 4)]

To create the users x products count table loop through the prodcut ids data to as a sparse matrix (much more memory efficient), column position contains the product ids with position listed in a dict.


In [8]:
indptr = [0]
indices = []
data = []
column_position = {}
# input must be a list of lists
for order in product_list['product_id']:
    for product in order:
        index = column_position.setdefault(product, len(column_position))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))
    
prod_matrix = csr_matrix((data, indices, indptr), dtype=int)
del(test)

In [17]:
prod_matrix.shape

The problem with using the sparse matrix is that it is gigantic, after all there are a lot of users and a lot of products. Non-negative matrix decomposition is implemented in sklearn, use that to decompose the count matrix into two new matrices with considerably reduced dimensions. 

In [9]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

nmf = NMF(n_components = 100, random_state = 42)
model = nmf.fit(prod_matrix)
H = model.components_
model.components_.shape

What can I do with the results? For one, I could pick a random user and find users that have similar purchasing behavior. 

In [10]:
W = model.transform(prod_matrix)
user_data = pd.DataFrame(normalize(W), index = product_list['user_id'])
idx = user_data.dot(user_data.iloc[0]).nlargest(5).index
user_data.dot(user_data.iloc[0]).nlargest(5)

In [11]:
def prod_count(product_ids):

    prod_count = {}
    for item in product_ids:
        if item not in prod_count.keys():
            prod_count[item] = 1
        elif item in prod_count.keys():
            prod_count[item] += 1
    return prod_count

In [12]:
similar_users = product_list[product_list.user_id.isin(idx)]
similar_users

In [40]:
overlap = set(similar_users.product_id.iloc[0]) & set(similar_users.product_id.iloc[2])

In [14]:
counts = similar_users.product_id.apply(prod_count)

Here are there order counts:

In [31]:
def id_values(row, overlap):
    for key, value in row.items():
        if key in overlap:
            print(key, value)

In [41]:
id_values(counts.iloc[0], overlap)

In [ ]:
id_values(counts.iloc[2], overlap)

Another benefit is that the W matrix contains latent information regarding the amount of each item a user has ordered in the past, but has vastly reduced dimensions. In other words, it doesn't keep each product as a column of the original user x product count matrix. I have yet to test this in creating predictions, but it should be a helpful way to keep track of user order counts per product. More to come!  